# SPTAG python wrapper tutorial 

To end-to-end build a vector search online service, it contains two steps:
- Offline build SPTAG index for database vectors
- Online serve the index to support vector search requests from the clients

## Offline build SPTAG index

> Prepare input vectors and metadatas for SPTAG

In [ ]:
import os
import shutil
import numpy as np

vector_number = 1000
vector_dimension = 100

# Randomly generate the database vectors. Currently SPTAG only support int8, int16 and float32 data type.
x = np.random.rand(vector_number, vector_dimension).astype(np.float32) 

# Prepare metadata for each vectors, separate them by '\n'. Currently SPTAG python wrapper only support '\n' as the separator
m = ''
for i in range(vector_number):
    m += str(i) + '\n'

> Build SPTAG index for database vectors **x**

In [7]:
import SPTAG

cmd = "./quantizer -d %d -v Float -f DEFAULT -i quan_idx/vectors.bin -o quan_doc_vectors.bin -oq quantizer.bin -qt PQQuantizer -qd %d -ts %d -norm false" % (vector_dimension, int(vector_dimension / 2), vector_number)


[1] Setting NumberOfThreads with value 4
[1] Setting DistCalcMethod with value L2
[1] Begin build index...


['graph.bin',
 'vectors.bin',
 'metadataIndex.bin',
 'deletes.bin',
 'tree.bin',
 'indexloader.ini',
 'metadata.bin']

[1] Start to build BKTree 1
[1] Lambda:min(1e-05,0.00309273) Max:96 Min:5 Avg:31.250000 Std/Avg:0.764910 Dist:12378.539062 NonZero/Total:32/32
[1] Lambda:min(0.000105263,0.0250586) Max:28 Min:1 Avg:2.968750 Std/Avg:1.578772 Dist:767.042969 NonZero/Total:32/32
[1] Lambda:min(0.00037037,0.22508) Max:1 Min:0 Avg:0.843750 Std/Avg:0.430331 Dist:0.000000 NonZero/Total:27/32
[1] 1 BKTree built, 1002 1000
[1] Build Tree time (s): 0
[1] build RNG graph!
[1] Parallel TpTree Partition begin
[1] Finish Getting Leaves for Tree 0
[1] Finish Getting Leaves for Tree 1
[1] Finish Getting Leaves for Tree 2
[1] Finish Getting Leaves for Tree 3
[1] Finish Getting Leaves for Tree 4
[1] Finish Getting Leaves for Tree 5
[1] Finish Getting Leaves for Tree 6
[1] Finish Getting Leaves for Tree 7
[1] Finish Getting Leaves for Tree 8
[1] Finish Getting Leaves for Tree 9
[1] Finish Getting Leaves for Tree 10
[1] Finish Getting Leaves for Tree 11
[1] Finish Getting Leaves for Tree 12
[1] Finish Getting Leaves for T

In [8]:
# Local index test on the vector search
index = SPTAG.AnnIndex.Load('sptag_index')

# prepare query vector
q = np.random.rand(vector_dimension).astype(np.float32)

result = index.SearchWithMetaData(q, 10) # Search k=3 nearest vectors for query vector q
print (result[0]) # nearest k vector ids
print (result[1]) # nearest k vector distances
print (result[2]) # nearest k vector metadatas

[1] Setting TreeFilePath with value tree.bin
[1] Setting GraphFilePath with value graph.bin
[1] Setting VectorFilePath with value vectors.bin
[1] Setting DeleteVectorFilePath with value deletes.bin
[1] Setting EnableBfs with value 0
[1] Setting BKTNumber with value 1
[1] Setting BKTKmeansK with value 32
[1] Setting BKTLeafSize with value 8
[1] Setting Samples with value 1000
[1] Setting BKTLambdaFactor with value 100.000000
[1] Setting TPTNumber with value 32
[1] Setting TPTLeafSize with value 2000
[1] Setting NumTopDimensionTpTreeSplit with value 5
[1] Setting NeighborhoodSize with value 32
[1] Setting GraphNeighborhoodScale with value 2.000000
[1] Setting GraphCEFScale with value 2.000000
[1] Setting RefineIterations with value 2
[1] Setting EnableRebuild with value 0
[1] Setting CEF with value 1000
[1] Setting AddCEF with value 500
[1] Setting MaxCheckForRefineGraph with value 8192
[1] Setting RNGFactor with value 1.000000
[1] Setting GPUGraphType with value 2
[1] Setting GPURefineS

> Build SPTAG index for database vector **x** with PQ quantization

In [13]:
import subprocess
import struct
import numpy as np

if (os.path.exists("quantizer.bin")):
    os.remove("quantizer.bin")
if (os.path.exists("quan_doc_vectors.bin")):
    os.remove("quan_doc_vectors.bin")
cmd = "Quantizer.exe -d %d -v Float -f DEFAULT -i sptag_index\\vectors.bin -o quan_doc_vectors.bin -oq quantizer.bin -qt PQQuantizer -qd %d -ts %d -norm false" % (vector_dimension, int(vector_dimension / 2), vector_number)
result = subprocess.run(cmd)
print (result) 
# For SPTAG index with quantization:
#     Cosine distance: norm -> true and use L2 for index build
#     L2 distance: norm -> false and use L2 for index build

f = open('quan_doc_vectors.bin', 'rb')
r = struct.unpack('i', f.read(4))[0]
c = struct.unpack('i', f.read(4))[0]
quan_x = np.frombuffer(f.read(r*c), dtype=np.uint8).reshape((r,c))
f.close()
print (quan_x.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'quantizer.bin -d 100 -v Float -f DEFAULT -i sptag_index\\vectors.bin -o quan_doc_vectors.bin -oq quantizer.bin -qt PQQuantizer -qd 50 -ts 1000 -norm false'

In [ ]:
import SPTAG

index = SPTAG.AnnIndex('BKT', 'UInt8', int(vector_dimension / 2))

# Set the thread number to speed up the build procedure in parallel 
index.SetBuildParam("NumberOfThreads", '4', "Index")

# Set the distance type. Currently SPTAG only support Cosine and L2 distances. Here Cosine distance is not the Cosine similarity. The smaller Cosine distance it is, the better.
index.SetBuildParam("DistCalcMethod", 'L2', "Index") 

if (os.path.exists("quan_sptag_index")):
    shutil.rmtree("quan_sptag_index")
if index.LoadQuantizer("quantizer.bin") and index.BuildWithMetaData(quan_x, m, vector_number, False, False):
    index.Save("quan_sptag_index") # Save the index to the disk

os.listdir('quan_sptag_index')

In [ ]:
# Local index test on the vector search
index = SPTAG.AnnIndex.Load('quan_sptag_index')
index.SetQuantizerADC(True)
result = index.SearchWithMetaData(q, 12) # Search k=3 nearest vectors for query vector q
print (result[0]) # nearest k vector ids
print (result[1]) # nearest k vector distances
print (result[2]) # nearest k vector metadatas

In [14]:
import SPTAG

index = SPTAG.AnnIndex('SPANN', 'Float', vector_dimension)

# Set the thread number to speed up the build procedure in parallel 
index.SetBuildParam("IndexAlgoType", "BKT", "Base")
index.SetBuildParam("IndexDirectory", "spann_index", "Base")
index.SetBuildParam("DistCalcMethod", "L2", "Base")

index.SetBuildParam("isExecute", "true", "SelectHead")
index.SetBuildParam("NumberOfThreads", "4", "SelectHead")
index.SetBuildParam("Ratio", "0.2", "SelectHead") # index.SetBuildParam("Count", "200", "SelectHead")

index.SetBuildParam("isExecute", "true", "BuildHead")
index.SetBuildParam("RefineIterations", "3", "BuildHead")
index.SetBuildParam("NumberOfThreads", "4", "BuildHead")

index.SetBuildParam("isExecute", "true", "BuildSSDIndex")
index.SetBuildParam("BuildSsdIndex", "true", "BuildSSDIndex")
index.SetBuildParam("PostingPageLimit", "12", "BuildSSDIndex")
index.SetBuildParam("SearchPostingPageLimit", "12", "BuildSSDIndex")
index.SetBuildParam("NumberOfThreads", "4", "BuildSSDIndex")
index.SetBuildParam("InternalResultNum", "32", "BuildSSDIndex")
index.SetBuildParam("SearchInternalResultNum", "64", "BuildSSDIndex")

if (os.path.exists("spann_index")):
    shutil.rmtree("spann_index")

if index.BuildWithMetaData(x, m, vector_number, False, False):
    index.Save("spann_index") # Save the index to the disk

os.listdir('spann_index')

[1] Setting IndexAlgoType with value BKT
[1] Setting IndexDirectory with value spann_index
[1] Setting DistCalcMethod with value L2
[1] Setting isExecute with value true
[1] Setting NumberOfThreads with value 4
[1] Setting Ratio with value 0.2
[1] Setting isExecute with value true
[1] Setting isExecute with value true
[1] Setting BuildSsdIndex with value true
[1] Setting PostingPageLimit with value 12
[1] Setting SearchPostingPageLimit with value 12
[1] Setting NumberOfThreads with value 4
[1] Setting InternalResultNum with value 32
[1] Setting SearchInternalResultNum with value 64
[1] Begin build index...
[1] Begin Select Head...
[1] Begin initial data (1000,100)...
[1] Begin Adjust Parameters...
[1] Start generating BKT.
[1] Start invoking BuildTrees.
[1] BKTKmeansK: 32, BKTLeafSize: 8, Samples: 1000, BKTLambdaFactor:-1.000000 TreeNumber: 1, ThreadNum: 4.
[1] Lambda:min(1,0.00210268) Max:83 Min:7 Avg:31.250000 Std/Avg:0.702817 Dist:12345.048828 NonZero/Total:32/32
[1] Lambda:min(0.1,

['HeadIndex',
 'SPTAGFullList.bin',
 'metadataIndex.bin',
 'SPTAGHeadVectorIDs.bin',
 'SPTAGHeadVectors.bin',
 'indexloader.ini',
 'metadata.bin']

In [ ]:
index = SPTAG.AnnIndex.Load('spann_index')
result = index.SearchWithMetaData(q, 12) # Search k=3 nearest vectors for query vector q
print (result[0]) # nearest k vector ids
print (result[1]) # nearest k vector distances
print (result[2]) # nearest k vector metadatas

In [ ]:
import SPTAG

index = SPTAG.AnnIndex('SPANN', 'UInt8', int(vector_dimension / 2))

# Set the thread number to speed up the build procedure in parallel 
index.SetBuildParam("IndexAlgoType", "BKT", "Base")
index.SetBuildParam("IndexDirectory", "spann_quan_index", "Base")
index.SetBuildParam("DistCalcMethod", "L2", "Base")
index.SetBuildParam("QuantizerFilePath", "quantizer.bin", "Base")

index.SetBuildParam("isExecute", "true", "SelectHead")
index.SetBuildParam("NumberOfThreads", "4", "SelectHead")
index.SetBuildParam("Ratio", "0.2", "SelectHead") # index.SetBuildParam("Count", "200", "SelectHead")

index.SetBuildParam("isExecute", "true", "BuildHead")
index.SetBuildParam("RefineIterations", "3", "BuildHead")
index.SetBuildParam("NumberOfThreads", "4", "BuildHead")

index.SetBuildParam("isExecute", "true", "BuildSSDIndex")
index.SetBuildParam("BuildSsdIndex", "true", "BuildSSDIndex")
index.SetBuildParam("PostingPageLimit", "12", "BuildSSDIndex")
index.SetBuildParam("SearchPostingPageLimit", "12", "BuildSSDIndex")
index.SetBuildParam("NumberOfThreads", "4", "BuildSSDIndex")
index.SetBuildParam("InternalResultNum", "32", "BuildSSDIndex")
index.SetBuildParam("SearchInternalResultNum", "64", "BuildSSDIndex")

if (os.path.exists("spann_quan_index")):
    shutil.rmtree("spann_quan_index")

if index.LoadQuantizer("quantizer.bin") and index.BuildWithMetaData(quan_x, m, vector_number, False, False):
    index.Save("spann_quan_index") # Save the index to the disk

os.listdir('spann_quan_index')

In [ ]:
index = SPTAG.AnnIndex.Load('spann_quan_index')
index.SetQuantizerADC(True)
result = index.SearchWithMetaData(q, 12) # Search k=3 nearest vectors for query vector q
print (result[0]) # nearest k vector ids
print (result[1]) # nearest k vector distances
print (result[2]) # nearest k vector metadatas

## Online serve the index

Start the vector search service on the host machine which listens for the client requests on the port 8000

> Write a server configuration file **service.ini** as follows:

```bash
[Service]
ListenAddr=0.0.0.0
ListenPort=8000
ThreadNumber=8
SocketThreadNumber=8

[QueryConfig]
DefaultMaxResultNumber=6
DefaultSeparator=|

[Index]
List=MyIndex

[Index_MyIndex]
IndexFolder=sptag_index
```

> Start the server on the host machine

```bash
Server.exe -m socket -c service.ini
```

It will print the follow messages:

```bash
Setting TreeFilePath with value tree.bin
Setting GraphFilePath with value graph.bin
Setting VectorFilePath with value vectors.bin
Setting DeleteVectorFilePath with value deletes.bin
Setting BKTNumber with value 1
Setting BKTKmeansK with value 32
Setting BKTLeafSize with value 8
Setting Samples with value 1000
Setting TPTNumber with value 32
Setting TPTLeafSize with value 2000
Setting NumTopDimensionTpTreeSplit with value 5
Setting NeighborhoodSize with value 32
Setting GraphNeighborhoodScale with value 2
Setting GraphCEFScale with value 2
Setting RefineIterations with value 2
Setting CEF with value 1000
Setting MaxCheckForRefineGraph with value 8192
Setting NumberOfThreads with value 4
Setting DistCalcMethod with value Cosine
Setting DeletePercentageForRefine with value 0.400000
Setting AddCountForRebuild with value 1000
Setting MaxCheck with value 8192
Setting ThresholdOfNumberOfContinuousNoBetterPropagation with value 3
Setting NumberOfInitialDynamicPivots with value 50
Setting NumberOfOtherDynamicPivots with value 4
Load Vector From sptag_index\vectors.bin
Load Vector (100, 10) Finish!
Load BKT From sptag_index\tree.bin
Load BKT (1,101) Finish!
Load Graph From sptag_index\graph.bin
Load Graph (100, 32) Finish!
Load DeleteID From sptag_index\deletes.bin
Load DeleteID (100, 1) Finish!
Start to listen 0.0.0.0:8000 ...
```

> Start python client to connect to the server and send vector search request.

In [ ]:
import SPTAGClient
import time

# connect to the server
client = SPTAGClient.AnnClient('127.0.0.1', '8000')
while not client.IsConnected():
    time.sleep(1)
client.SetTimeoutMilliseconds(18000)

k = 3
vector_dimension = 10
# prepare query vector
q = np.random.rand(vector_dimension).astype(np.float32)

result = client.Search(q, k, 'Float', True) # AnnClient.Search(query_vector, knn, data_type, with_metadata)

print (result[0]) # nearest k vector ids
print (result[1]) # nearest k vector distances
print (result[2]) # nearest k vector metadatas

